# BLAST in-scanner behavioral analysis

Violet Kozloff

Last updated: May 29th 2019 

This script analyzes structured and random blocks across four tasks: auditory (speech and tones) and visual (letters and images).

It measures the mean reaction time and the slope of the reaction time for each participant for each condition.

# Extract and prepare data

## Prepare environment

Load packages

In [18]:
install.packages("dplyr")
install.packages("reshape")
library ("plyr")
library("reshape")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


Set working directory

In [19]:
setwd("/Volumes/data/projects/blast/data/mri/in_scanner_behavioral/adult/sl_raw_data")

ERROR: Error in setwd("/Volumes/data/projects/blast/data/mri/in_scanner_behavioral/adult/sl_raw_data"): cannot change working directory


Remove objects in environment

In [21]:
rm(list=ls())


## Extract auditory data

List files

In [22]:
auditory_files <- list.files(pattern=glob2rx("*auditory_*.csv"))


Extract only the relevant columns of auditory data
